In [ ]:
#!pip install idx2numpy

import os 
import idx2numpy
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm


if not os.path.isfile("./MNIST.tar.gz"):
    print("downloading MNIST zip data")
    os.system("wget www.di.ens.fr/~lelarge/MNIST.tar.gz")
if not os.path.exists("MNIST"):
    print("Unzipping files")
    os.system("tar -zxvf MNIST.tar.gz")
print("Done")

train_x = idx2numpy.convert_from_file("MNIST/raw/train-images-idx3-ubyte")
train_y = idx2numpy.convert_from_file("MNIST/raw/train-labels-idx1-ubyte")

test_x = idx2numpy.convert_from_file("MNIST/raw/t10k-images-idx3-ubyte")
test_y = idx2numpy.convert_from_file("MNIST/raw/t10k-labels-idx1-ubyte")

# preprocess the data

TRAIN_X, TRAIN_Y, TEST_X, TEST_Y = [], [], [], []
for idx, label in enumerate(train_y):
    TRAIN_X.append(train_x[idx].reshape(-1, )/255.0)
    TRAIN_Y.append(label)
    
for idx, label in enumerate(test_y):
    TEST_X.append(test_x[idx].reshape(-1, )/255.0)
    TEST_Y.append(label)
    
TRAIN_X = np.array(TRAIN_X)
TRAIN_Y = np.array(TRAIN_Y)
TEST_X = np.array(TEST_X)
TEST_Y = np.array(TEST_Y)

device = 'cuda' if torch.cuda.is_available() else 'cpu'


class Network(nn.Module):

    def __init__(self):
        super().__init__()

        self.fc1 = nn.Linear(784, 32)
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self,x):
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)

        return x

def train(X, y, bs, epochs):
    
    # X --> Input.
    # y --> true/target value.
    # bs --> Batch Size.
    # epochs --> Number of iterations.
    # lr --> Learning rate. 
    # m-> number of training examples
    # n-> number of features 
    m, n = X.shape
    
    net = Network().to(device)
    optm = Adam(net.parameters(), lr = 0.001)
    criterion = nn.CrossEntropyLoss()

    losses = []
    train_acc = []
    test_acc = []
    # Training loop.
    #print("comes in training loop")
    pbar =tqdm(range(epochs))
    for epoch in pbar:
        epoch_loss = 0
        for i in range((m-1)//bs + 1):
            start_i = i*bs
            end_i = start_i + bs
            xb = X[start_i:end_i]
            yb = y[start_i:end_i]
            #print("in batch calculation")
            net.zero_grad()
            output = net(torch.from_numpy(xb).float().to(device))
            
            loss=criterion(output, torch.from_numpy(yb).to(device))
            epoch_loss+=loss.item()
            loss.backward()
            optm.step()
        print("loss = ", epoch_loss)
            
            
    '''
        #tr_acc = accuracy_score(TRAIN_Y, predict(TRAIN_X, w))
        #te_acc = accuracy_score(TEST_Y, predict(TEST_X, w))
        #losses.append(l)
        #train_acc.append(tr_acc)
        #test_acc.append(te_acc)
        #print("loss = ", l)
        #print("training accuracy ", tr_acc)
        #print("testing accuracy ", te_acc)
        
    # returning weights, bias and losses(List).
    #return w,losses, train_acc, test_acc
    '''


def predict(X, w):
    # X --> Input.
    
    # Calculating presictions/y_hat.
    preds = sigmoid(np.dot(X, w))
    pred_class = []
    pred_class = [1 if i > 0.5 else 0 for i in preds]
    
    return np.array(pred_class)

'''
w, l, train_acc, test_acc = train(TRAIN_X, TRAIN_Y, bs=256, epochs=40)
plt.plot([i for i in range(1, 101)], train_acc, label = "Training Accuracy")
plt.plot([i for i in range(1, 101)], test_acc, label = "Testing Accuracy")
plt.legend()
plt.savefig('train_test.png', dpi=300)
plt.clf()
plt.plot([i for i in range(1, 101)], l, label = "Log Liklihood")
plt.legend()
plt.savefig('log_liklihood.png', dpi=300)
plt.clf()
#print(TRAIN_X[1])
#print(TRAIN_X.shape)
'''

Done


'\nw, l, train_acc, test_acc = train(TRAIN_X, TRAIN_Y, bs=256, epochs=40)\nplt.plot([i for i in range(1, 101)], train_acc, label = "Training Accuracy")\nplt.plot([i for i in range(1, 101)], test_acc, label = "Testing Accuracy")\nplt.legend()\nplt.savefig(\'train_test.png\', dpi=300)\nplt.clf()\nplt.plot([i for i in range(1, 101)], l, label = "Log Liklihood")\nplt.legend()\nplt.savefig(\'log_liklihood.png\', dpi=300)\nplt.clf()\n#print(TRAIN_X[1])\n#print(TRAIN_X.shape)\n'

In [ ]:
train(TRAIN_X, TRAIN_Y, bs=256, epochs=40)

  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  2%|▎         | 1/40 [00:00<00:21,  1.77it/s]

loss =  326.7388833165169


  5%|▌         | 2/40 [00:01<00:21,  1.75it/s]

loss =  143.61013171076775


  8%|▊         | 3/40 [00:01<00:21,  1.74it/s]

...

loss =  22.152578975073993


 98%|█████████▊| 39/40 [00:22<00:00,  1.77it/s]

loss =  21.71517027914524


100%|██████████| 40/40 [00:22<00:00,  1.77it/s]

loss =  21.291793935000896


In [ ]:

# logistic regression for multi-class classification using built-in one-vs-rest
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define model
model = LogisticRegression(multi_class='ovr')
# fit model
model.fit(X, y)
# make predictions
yhat = model.predict(X)

In [ ]:
make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)


(array([[ 1.89149379, -0.39847585,  1.63856893, ...,  0.58810926,
         -0.02542177, -0.52835426],
        [ 1.86913773, -0.56336215,  2.03411807, ..., -0.55633   ,
         -0.27340013,  0.72129251],
        [ 1.95259701, -2.83144572,  0.19055535, ..., -0.60957741,
         -2.07750191, -1.75469982],
        ...,
        [ 1.98033054,  1.37228804,  1.67376262, ...,  2.75687956,
         -0.20719842,  0.39299534],
        [-1.54305631, -0.27530218,  0.38744703, ...,  0.72769107,
          0.51983329, -1.92306657],
        [ 1.17970389, -3.3812155 , -0.1498426 , ...,  1.24640268,
         -4.00356845, -3.01027048]]),
 array([1, 0, 1, 2, 0, 2, 2, 2, 0, 0, 1, 0, 2, 2, 0, 1, 1, 0, 1, 0, 2, 0,
        2, 2, 0, 0, 2, 0, 1, 1, 2, 1, 0, 2, 0, 0, 0, 1, 2, 2, 2, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 1,
        1, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 0, 1, 2, 2, 2,
        0, 1, 0, 2, 1, 0, 2, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2,
   

In [ ]:
# example of a multi-label classification task
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
# define dataset
X, y = make_multilabel_classification(n_samples=2000, n_features=10, n_classes=3, n_labels=2, random_state=1)
# summarize dataset shape
print(X.shape, y.shape)
# summarize first few examples
for i in range(10):
	print(X[i], y[i])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(2000, 10) (2000, 3)
[ 3.  3.  6.  7.  8.  2. 11. 11.  1.  3.] [1 1 0]
[7. 6. 4. 4. 6. 8. 3. 4. 6. 4.] [0 0 0]
[ 5.  5. 13.  7.  6.  3.  6. 11.  4.  2.] [1 1 0]
[1. 1. 5. 5. 7. 3. 4. 6. 4. 4.] [1 1 1]
[ 4.  2.  3. 13.  7.  2.  4. 12.  1.  7.] [0 1 0]
[ 4.  3.  3.  2.  5.  2.  3.  7.  2. 10.] [0 0 0]
[ 3.  3.  3. 11.  6.  3.  4. 14.  1.  3.] [0 1 0]
[ 2.  1.  7.  8.  4.  5. 10.  4.  6.  6.] [1 1 1]
[ 5.  1.  9.  5.  3.  4. 11.  8.  1.  8.] [1 1 1]
[ 2. 11.  7.  6.  2.  2.  9. 11.  9.  3.] [1 1 1]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in range(3):
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train,[y[category] for y in y_train])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score([y[category] for y in y_test], prediction)))
    print("\n")

**Processing 0 comments...**
Test accuracy is 0.8875


**Processing 1 comments...**
Test accuracy is 0.84


**Processing 2 comments...**
Test accuracy is 0.87




In [ ]:
# using binary relevance
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier
classifier = BinaryRelevance(LogisticRegression())#,GaussianNB
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))


Accuracy =  0.685


In [ ]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.79




In [ ]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(X_train, y_train)
# predict
predictions = classifier.predict(X_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")


Accuracy =  0.83




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test).toarray()
# train
classifier_new.fit(x_train, y_train)
# predict
predictions_new = classifier_new.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_new))


TypeError: ignored